# Assignment 1: Introduction to the Fully Recurrent Network

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.


## Exercise 1: Numerical stability of the binary cross-entropy loss function

We will use the binary cross-entropy loss function to train our RNN, which is defined as 
$$
L_{\text{BCE}}(\hat y, y) = -y \log \hat y - (1-y) \log (1-\hat y),
$$
where $y$ is the label and $\hat y$ is a prediction, which comes from a model (e.g. an RNN) and is usually sigmoid-activated, i.e., we have
$$
\hat y = \sigma(z) = \frac{1}{1+e^{|-z|}}.
$$
The argument $z$ is called *logit*. For reasons of numerical stability it is better to let the model emit the logit $z$ (instead of the prediction $\hat y$) and incorporate the sigmoid activation into the loss function. Explain why this is the case and how we can gain numerical stability by combining the two functions $L_{\text{BCE}}(\hat y, y)$ and $\sigma(z)$ into one function $L(z, y) = L_{\text{BCE}}(\sigma(z), y)$. 

*Hint: Prove that $\log(1+e^{z}) = \log (1+e^{-|z|}) + \max(0, z)$ and argue why the right-hand side is numerically more stable. Finally, express $L(z,y)$ in terms of that form.*

########## YOUR SOLUTION HERE ##########

>Prove that $\log(1+e^{z}) = \log (1+e^{-|z|}) + \max(0, z)$

Let's do it:
$$
\log(1+e^{z}) = \log (1+e^{-|z|}) + \max(0, z)
$$
remember, that $e^{|-z|} = \frac{1}{e^{z}}$ and $\log(\frac{a}{b}) = \log(a)-\log(b)$ , so  
$$
\log (1+e^{-|z|}) = \log (1+ \frac{1}{e^{z}}) = \log (\frac{e^{z}+1}{e^{z}})= \log(e^{z}+1) - \log(e^{z}) =\log(e^{z}+1) - {z} 
$$

Substituting $\log (1+e^{-|z|})$ in the original expression through the calculation above results in the following:
$$
\log(1+e^{z}) = \log(e^{z}+1) - {z} + \max(0,z)
$$
assuming that $\max(0,z)$ results in most cases in $z$, so $- {z} + \max(0,z)$ are excluding each other, what results in:
$$
\log(1+e^{z}) = \log(e^{z}+1)
$$

>argue why the right-hand side is numerically more stable

Right-hand side is numerically more stable due to its linearity element expressed through $\max(0, z)$.

>Finally, express $L(z,y)$ in terms of that form.

Now let's completely express BCE loss function in terms of $\hat y = \sigma(z) = \frac{1}{1+e^{|-z|}}$:
$$
L_{\text{BCE}}(z, y) = -y \log( \frac{1}{1+e^{|-z|}}) - (1-y) \log (1-\frac{1}{1+e^{|-z|}}) = 
$$

$$
=-y(\log(1)-\log(1+e^{|-z|}))-(1-y)\log(\frac{1+e^{|-z|}-1}{1+e^{|-z|}})=
$$

$$
= -y(0-\log(1+e^{|-z|}))-(1-y)\log(\frac{e^{|-z|}}{1+e^{|-z|}}) =
$$

$$
= -y(-\log(1+e^{|-z|}))-(1-y)(\log(e^{|-z|}) - \log(1+e^{|-z|}))= 
$$

$$
= -y(-\log(1+e^{|-z|}))-(1-y)(-z - \log(1+e^{|-z|}))
$$
let's express $\log(1+e^{|-z|})$ so that $-z$ will become positive:
$$
-y(-\log(1+e^{z})+z)-(1-y)(-z - \log(1+e^{z})+z)=  
$$
$$
=-y(-\log(1+e^{z})+z)-(1-y)(- \log(1+e^{z}))
$$
Now we can susbtitute $\log(1+e^{z})$ through the expression above given in the hint:
$$
=-y(-\log (1+e^{-|z|}))-(1-y)(- \log (1+e^{-|z|}) - \max(0, z))=
$$
$$
=-y(-\log(1+e^{-|z|}))+(1-y)(\log(1+e^{-|z|})+\max(0, z))=
$$
$$
=y\log(1+e^{-|z|})+\log(1+e^{-|z|})-y\log(1+e^{-|z|})+\max(0, z)-y\max(0, z)=
$$
$$
=\log(1+e^{-|z|})+\max(0, z)(1-y)
$$


## Exercise 2: Derivative of the loss

Calculate the derivative of the binary cross-entropy loss function $L(z, y)$ with respect to the logit $z$.

########## YOUR SOLUTION HERE ##########

<!-- $\frac{\partial f}{\partial x}$ -->
$$
\frac{\partial L_{\text{BCE}}(z, y)}{\partial z}=\frac{\log(1+e^{-|z|})+\max(0, z)(1-y)}{\partial z} = 
$$

$$
=\frac{\log(1+e^{-|z|})+\max(0, z)(1-y)}{\partial z}=
$$
$$
=\frac{\partial \log(1+e^{-|z|})}{\partial z}+(1-y)=
$$

$$
=\frac{\partial \log(1+e^{z})}{\partial z} - \frac{\partial \log(e^z)}{z}+(1-y)=
$$

$$
=\frac{e^z}{1+e^z} -y
$$

## Exercise 3: Initializing the network
Consider the fully recurrent network
$$
s(t) = W x(t) + R a(t-1) \\
a(t) = \tanh(s(t)) \\
z(t) = V a(t) \\
\hat y(t) = \sigma(z(t))
$$
for $t \in \mathbb{N}, x(t) \in \mathbb{R}^{D}, s(t) \in \mathbb{R}^{I}, a(t) \in \mathbb{R}^{I}, z(t) \in \mathbb{R}^K, \hat y(t) \in \mathbb{R}^K$ and $W, R, V$ are real matrices of appropriate sizes and $\hat a(0) = 0$. 

*Compared to the lecture notes we choose $f(x) = \tanh(x) = (e^x - e^{-x})(e^x + e^{-x})^{-1}$ and $\varphi(x) = \sigma(x) = (1+e^{-x})^{-1}$. Further, we introduced an auxiliary variable $z(t)$ and transposed the weight matrices.*

Write a function `init` that takes a `model` and integers $D, I, K$ as arguments and stores the matrices $W, R, V$ as members `model.W`, `model.R`, `model.V`, respectively. The matrices should be `numpy` arrays of appropriate sizes and filled with random values that are uniformly distributed between -0.01 and 0.01. 

>The matrices should be `numpy` arrays of appropriate sizes and filled with random values that are uniformly distributed between -0.01 and 0.01. 

From the lecture notes (page 27) the dimensions of matrices should be set as following: $W \in \mathbb{R}^{D \times I}$, $R \in \mathbb{R}^{I \times I}$, $V \in \mathbb{R}^{I \times K}$

In [6]:
%matplotlib inline
import numpy as np
from scipy.special import expit as sigmoid

class Obj(object):
    pass

model = Obj()
T, D, I, K = 10, 3, 5, 1

def init(model, D, I, K):
    ########## YOUR SOLUTION HERE ##########
    model.W = np.random.uniform(low=-0.01, high=0.01,size=(D,I))#input weight matrix: maps input features to hidden representations
    model.R = np.random.uniform(low=-0.01, high=0.01,size=(I,I))#recurrent weight matrix: holds the weights of the loop connections, maps hidden representations forward in time.
    model.V = np.random.uniform(low=-0.01, high=0.01,size=(I,K))#output weight matrix which maps the network’s hidden representations to the output/target space.

Obj.init = init
model.init(D, I, K)

let'see whether it works as desired:

In [14]:
print(f"W: {model.W}\n W's shape: {model.W.shape}\n\nR: {model.R}\n R's shape: {model.R.shape}\n\nV: {model.V}\n V's shape: {model.V.shape}")

W: [[ 0.00416206 -0.00424522  0.00723071  0.00801127  0.00554817]
 [-0.00970395 -0.00809582  0.00701223 -0.00739459  0.00281504]
 [-0.00577162  0.00996883  0.0060737   0.0025208   0.0068755 ]]
 W's shape: (3, 5)

R: [[ 0.00738186 -0.00390577  0.00092534 -0.0072466   0.00389606]
 [ 0.00240492 -0.00469909  0.00428711 -0.00734574 -0.00731057]
 [-0.0088957   0.00486529  0.00318953  0.00698348  0.00387764]
 [ 0.0019952   0.00328065 -0.00362756 -0.0055909   0.00105359]
 [-0.00605044  0.00816233 -0.00701411 -0.00505808 -0.00100771]]
 R's shape: (5, 5)

V: [[ 0.00342739]
 [ 0.00658612]
 [-0.0012201 ]
 [ 0.0099079 ]
 [-0.00056608]]
 V's shape: (5, 1)


## Exercise 4: The forward pass
Implement the forward pass for the fully recurrent network for sequence classification (many-to-one mapping). To this end, write a function `forward` that takes a `model`, a sequence of input vectors `x`, and a label `y` as arguments. The inputs will be represented as a `numpy` array of shape `(T, D)`. It should execute the behavior of the fully recurrent network and evaluate the (numerically stabilized) binary cross-entropy loss at the end of the sequence and return the resulting loss value. Store the sequence of hidden activations $(a(t))_{t=1}^T$ and the logit $z(T)$ into `model.a` and `model.z`, respectively. 

In [35]:

def forward(model, x, y):
    model.x = x
    model.y = y
    model.a = np.zeros((T,I)) # a ∈ ℝ^(10x5)
    model.z = np.zeros(T) # z ∈ ℝ^(10,)
    loss = np.empty(T) # z ∈ ℝ^(10,)
    """
    Often, a(0) = 0 is a reasonable choice as this corresponds to a “clean” memory.    
    """
    for t in range (T): # i.e. in range(10)
        
        if t==0:
            #use the current recurrent element, since it's zero
            a_t = model.a[t].reshape(model.a.shape[1],1) # a_t ∈ ℝ^(5x1)
        else:
            #now use the previous recurrent element 
            a_t = model.a[t-1].reshape(model.a.shape[1],1) # a_t ∈ ℝ^(5x1)    

        w_T = np.transpose(model.W) # w_T ∈ ℝ^(5x3)
        x_t = model.x[t].reshape(model.x.shape[1],1) # x_t ∈ ℝ^(3x1)

        #     (5,3)x(3,1)=(5,1)
        s_t = np.dot(w_T,x_t) + \
              np.dot(model.R,a_t)
        #     (5,5)x(5,1)=(5,1)  
        # ==> s_t ∈ ℝ^(5x1)

        a_t = np.tanh(s_t) # same dims as with s_t
        v_T = np.transpose(model.V) # v_T ∈ ℝ^(1x5)
        model.z[t] = np.dot(v_T,a_t) # model.z[t] ∈ ℝ^(1x1)
    
        loss[t] = np.log(1+np.exp(-np.absolute(model.z[t])))+max(0,model.z[t].item())*(1-y)
        model.a[t] = a_t.reshape(1,I) # add the recurrent node
    return loss[-1]

Obj.forward = forward
model.forward(np.random.uniform(-1, 1, (T, D)), 1)

[0.69310711 0.69310996 0.69303791 0.69307281 0.69309539 0.69310193
 0.69313186 0.69306762 0.69306673 0.69306729]


0.6930672858457247

## Exercise 5: The computational graph

Visualize the computational graph of the fully recurrent network unfolded in time. The graph should show the functional dependencies of the nodes $x(t), a(t), z(t), L(z(t), y(t))$ for $t \in \{1, 2, 3\}$. Use the package `networkx` in combination with `matplotlib` to draw a directed graph with labelled nodes and edges. If you need help take a look at [this guide](https://networkx.guide/visualization/basics/). Make sure to arrange the nodes in a meaningful way. 

In [1]:
import networkx as nx
import matplotlib.pyplot as plt

########## YOUR SOLUTION HERE ##########